# AI-PSCI-013: ESM-1v for Variant Effect Prediction

**AI in Pharmaceutical Sciences: Bench to Bedside**  
VCU School of Pharmacy | VIP Program | Spring 2026

---

**Week 7 | Module: AI Tools | Estimated Time: 60-90 minutes**

**Prerequisites**: AI-PSCI-001 through AI-PSCI-012

**GPU Recommended**: ESM-1v runs faster with GPU but works on CPU.

---

## 🎯 Learning Objectives

After completing this talktorial, you will be able to:

1. Use ESM-1v to score mutation effects on protein function
2. Interpret log-likelihood ratios (LLRs) for variant assessment
3. Identify potentially deleterious mutations computationally
4. Compare ESM-1v predictions with experimental data
5. Apply evolutionary conservation principles to drug resistance

---

## 📚 Background

### Protein Language Models for Variant Effect Prediction

**ESM-1v** (Evolutionary Scale Modeling - 1 variant) is a protein language model developed by Meta AI that can predict the functional effects of mutations **without requiring structural information**. It was trained on 250 million protein sequences and learns evolutionary constraints.

### The Core Insight

Protein language models learn which amino acids are "expected" at each position based on evolutionary patterns. If a mutation replaces a conserved residue with something unusual, the model assigns a low probability to that mutation — suggesting it may be deleterious.

### Log-Likelihood Ratio (LLR)

The key metric is the **log-likelihood ratio**:

```
LLR = log P(mutant amino acid) - log P(wild-type amino acid)
```

**Interpretation**:
- **LLR ≈ 0**: Mutation is neutral (equally likely as wild-type)
- **LLR < 0**: Mutation is disfavored (potentially deleterious)
- **LLR > 0**: Mutation is favored (rare, may be beneficial)
- **LLR < -2**: Strongly disfavored (likely deleterious)

### Zero-Shot Prediction

ESM-1v can make predictions without any target-specific training data — this is called "zero-shot" prediction. The model uses only the protein sequence and its learned evolutionary knowledge.

### Masked Marginal Scoring

The scoring works by:
1. Masking one position at a time in the sequence
2. Predicting the probability distribution over all 20 amino acids
3. Comparing the probability of wild-type vs mutant

### Applications in Drug Discovery

- **Resistance prediction**: Identify mutations likely to cause drug resistance
- **Protein engineering**: Guide rational design of improved variants
- **Clinical interpretation**: Assess pathogenicity of patient variants
- **Target validation**: Understand functional constraints on drug targets

---

## 🛠️ Setup

Run this cell to install required packages. **This takes 2-3 minutes.**

In [ ]:
#@title 🛠️ Install Packages (2-3 min)
import sys
import subprocess

# Check for GPU
try:
    gpu_info = !nvidia-smi --query-gpu=name --format=csv,noheader
    if gpu_info and gpu_info[0] and 'NVIDIA' in gpu_info[0]:
        print(f"✅ GPU detected: {gpu_info[0]}")
        print("   ESM-1v will run much faster with GPU.")
        HAS_GPU = True
    else:
        print("⚠️ No GPU detected. ESM-1v will run on CPU (slower).")
        HAS_GPU = False
except Exception:
    print("⚠️ Could not detect GPU. ESM-1v will run on CPU.")
    HAS_GPU = False

# Install PyTorch (GPU or CPU version)
print("\n📦 Installing PyTorch...")
if HAS_GPU:
    !pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121 -q
else:
    !pip install torch torchvision -q

# Install ESM
print("📦 Installing ESM (fair-esm)...")
!pip install fair-esm -q

# Install other dependencies
print("📦 Installing analysis packages...")
!pip install biopython matplotlib seaborn pandas numpy requests -q

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

print("\n✅ All packages installed!")

Import required libraries:

In [ ]:
#@title 📦 Import Libraries
import torch
import esm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from pathlib import Path
from typing import List, Dict, Tuple
import time

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️ Using device: {device}")

# Set display options
pd.set_option('display.max_columns', 10)
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

print("✅ All libraries imported!")

---

## 🎯 Target Configuration

Select your drug target. This selection determines which protein you'll analyze.

In [ ]:
#@title 🎯 Select Your Drug Target
TARGET = "DHFR" #@param ["DHFR", "ABL1", "EGFR", "AChE", "COX-2", "DPP-4"]

TARGET_CONFIG = {
    "DHFR": {
        "pdb": "1RX1", "uniprot": "P0ABQ4", "drug": "Trimethoprim", "organism": "E. coli",
        "mutations": ["L28R", "P21L", "A26T", "W30R", "I94L"],
        "mutation_context": "Resistance mutations reducing trimethoprim binding",
        "expected_length": 159,
        "clinical_mutations": {"L28R": "resistant", "P21L": "resistant", "A26T": "resistant", 
                               "W30R": "resistant", "I94L": "resistant"}
    },
    "ABL1": {
        "pdb": "1IEP", "uniprot": "P00519", "drug": "Imatinib", "organism": "Human",
        "mutations": ["T315I", "E255K", "Y253H", "G250E", "M351T"],
        "mutation_context": "Resistance mutations causing imatinib resistance in CML",
        "expected_length": 1130, "domain_start": 242, "domain_end": 493,
        "clinical_mutations": {"T315I": "resistant", "E255K": "resistant", "Y253H": "resistant",
                               "G250E": "resistant", "M351T": "resistant"}
    },
    "EGFR": {
        "pdb": "1M17", "uniprot": "P00533", "drug": "Erlotinib", "organism": "Human",
        "mutations": ["T790M", "L858R", "C797S", "G719S", "L861Q"],
        "mutation_context": "Drug resistance and activating mutations in lung cancer",
        "expected_length": 1210, "domain_start": 712, "domain_end": 979,
        "clinical_mutations": {"T790M": "resistant", "L858R": "activating", "C797S": "resistant",
                               "G719S": "activating", "L861Q": "activating"}
    },
    "AChE": {
        "pdb": "4EY7", "uniprot": "P22303", "drug": "Donepezil", "organism": "Human",
        "mutations": ["Y337A", "F338A", "W286A", "E202Q", "S203A"],
        "mutation_context": "Mutations in catalytic gorge affecting substrate binding",
        "expected_length": 614,
        "clinical_mutations": {"Y337A": "functional", "F338A": "functional", "W286A": "functional",
                               "E202Q": "catalytic", "S203A": "catalytic"}
    },
    "COX-2": {
        "pdb": "3LN1", "uniprot": "P35354", "drug": "Celecoxib", "organism": "Human",
        "mutations": ["V523I", "S530A", "Y385F", "R120A", "H90Q"],
        "mutation_context": "Mutations affecting selectivity and catalysis",
        "expected_length": 604,
        "clinical_mutations": {"V523I": "selectivity", "S530A": "catalytic", "Y385F": "catalytic",
                               "R120A": "substrate", "H90Q": "functional"}
    },
    "DPP-4": {
        "pdb": "1X70", "uniprot": "P27487", "drug": "Sitagliptin", "organism": "Human",
        "mutations": ["S630A", "E205A", "E206A", "Y547A", "R125A"],
        "mutation_context": "Catalytic and binding site mutations",
        "expected_length": 766,
        "clinical_mutations": {"S630A": "catalytic", "E205A": "substrate", "E206A": "substrate",
                               "Y547A": "binding", "R125A": "binding"}
    }
}

config = TARGET_CONFIG[TARGET]
print(f"✅ Target: {TARGET}")
print(f"   PDB: {config['pdb']} | UniProt: {config['uniprot']}")
print(f"   Reference Drug: {config['drug']}")
print(f"   Organism: {config['organism']}")
print(f"\n🧬 Mutations to analyze: {', '.join(config['mutations'])}")
print(f"   Context: {config['mutation_context']}")

# Create output directory
output_dir = Path(f"esm1v_{TARGET}")
output_dir.mkdir(exist_ok=True)
print(f"\n📁 Output directory: {output_dir}")

---

## 🔬 Guided Inquiry 1: Loading ESM-1v Model

### Context

ESM-1v comes as an ensemble of 5 models trained with different random seeds. Using multiple models improves prediction robustness. We'll start by loading one model and understanding its architecture.

### Your Task

Using your AI assistant, write code to:

1. Load the ESM-1v model from the esm library
2. Understand the model architecture (layers, parameters)
3. Get the alphabet (vocabulary) for amino acids
4. Move the model to GPU if available

💡 **Prompting Tips**:
- Ask: "How do I load ESM-1v from fair-esm library?"
- Ask: "What is the batch converter and how is it used?"
- The model names are: `esm1v_t33_650M_UR90S_1` through `esm1v_t33_650M_UR90S_5`

### Verification

After running your code, confirm:
- [ ] Model loads successfully
- [ ] Model is on GPU (if available)
- [ ] Alphabet contains 20 standard amino acids

📓 **Lab Notebook**: Record the model architecture. How many parameters does ESM-1v have?

In [ ]:
# Your code here


---

## 🔬 Guided Inquiry 2: Fetching and Preparing Sequence

### Context

Before scoring mutations, we need the wild-type protein sequence. We'll fetch it from UniProt and prepare it for ESM-1v.

### Your Task

Using your AI assistant, write code to:

1. Fetch the protein sequence from UniProt
2. For kinases, extract the relevant domain
3. Tokenize the sequence for ESM-1v
4. Verify the sequence matches expected length

💡 **Prompting Tips**:
- Ask: "How do I tokenize a protein sequence for ESM?"
- The batch_converter takes a list of (name, sequence) tuples
- Output tokens have <cls> at start and <eos> at end

### Verification

After running your code, confirm:
- [ ] Sequence is fetched successfully
- [ ] Sequence length matches expected
- [ ] Tokens are correctly shaped: [1, seq_len + 2]

📓 **Lab Notebook**: Record the sequence length and first/last 10 amino acids.

In [ ]:
# Your code here


---

## 🔬 Guided Inquiry 3: Computing Mutation Log-Likelihoods

### Context

The core of ESM-1v variant effect prediction is masked marginal scoring. We mask one position at a time and get the model's predicted probabilities for all 20 amino acids.

### Your Task

Using your AI assistant, write code to:

1. Mask a single position in the sequence
2. Get the model's logits (unnormalized predictions)
3. Convert logits to probabilities with softmax
4. Calculate log-likelihood ratio for a mutation

💡 **Prompting Tips**:
- Ask: "How do I mask a position in ESM tokenized sequence?"
- The mask token index is `alphabet.mask_idx`
- Position in tokens = position in sequence + 1 (for <cls>)

### Verification

After running your code, confirm:
- [ ] Logits have shape [1, seq_len+2, vocab_size]
- [ ] Probabilities sum to 1 at each position
- [ ] LLR is calculated correctly

📓 **Lab Notebook**: For one example mutation, record the WT probability, mutant probability, and LLR.

In [ ]:
# Your code here


---

## 🔬 Guided Inquiry 4: Scoring All Target Mutations

### Context

Now let's score all the clinically relevant mutations for our target using ESM-1v. This will give us a comprehensive view of predicted mutation effects.

### Your Task

Using your AI assistant, write code to:

1. Loop through all mutations in `config['mutations']`
2. Calculate LLR for each mutation
3. Store results in a DataFrame
4. Classify mutations by predicted effect

💡 **Prompting Tips**:
- Ask: "How do I efficiently score multiple mutations?"
- Consider caching logits for positions with multiple mutations
- Add error handling for sequence mismatches

### Verification

After running your code, confirm:
- [ ] All mutations are scored
- [ ] Results are stored in a DataFrame
- [ ] LLR values are reasonable (typically -5 to +2)

📓 **Lab Notebook**: Record the LLR for each mutation. Which has the most negative LLR?

In [ ]:
# Your code here


---

## 🔬 Guided Inquiry 5: Visualizing Mutation Effects

### Context

Visualization helps us understand mutation effects at a glance. We'll create plots showing LLR distributions and compare predictions across mutations.

### Your Task

Using your AI assistant, write code to:

1. Create a bar plot of LLRs for all mutations
2. Color-code by predicted effect (deleterious vs neutral)
3. Add a heatmap of position-specific amino acid preferences
4. Annotate with clinical information if available

💡 **Prompting Tips**:
- Ask: "How do I create a diverging bar chart with positive/negative values?"
- Use red for deleterious (negative LLR), green for neutral/positive
- Consider adding a reference line at LLR = 0

### Verification

After running your code, confirm:
- [ ] Bar plot clearly shows LLR values
- [ ] Colors match predictions
- [ ] Clinical annotations are visible

📓 **Lab Notebook**: Which mutations are predicted most deleterious? Do these match known clinical data?

In [ ]:
# Your code here


---

## 🔬 Guided Inquiry 6: Ensemble Scoring for Robustness

### Context

ESM-1v comes as an ensemble of 5 models. Using multiple models and averaging their predictions improves robustness and reliability. Let's implement ensemble scoring.

### Your Task

Using your AI assistant, write code to:

1. Load multiple ESM-1v models (3 or 5)
2. Score mutations with each model
3. Average the LLRs across models
4. Calculate standard deviation as uncertainty estimate

💡 **Prompting Tips**:
- Ask: "How do I load multiple ESM-1v models for ensemble prediction?"
- Models are named: `esm1v_t33_650M_UR90S_1` through `esm1v_t33_650M_UR90S_5`
- Consider memory constraints — clear models after use if needed

### Verification

After running your code, confirm:
- [ ] Multiple models are loaded and used
- [ ] Average LLR is computed
- [ ] Standard deviation is reasonable (typically 0.1-0.5)

📓 **Lab Notebook**: How much do predictions vary across models? Which mutations have highest uncertainty?

In [ ]:
# Your code here


In [ ]:
# Your code here


---

## ✅ Checkpoint

Before moving on to the next talktorial, confirm you can:

- [ ] Load ESM-1v model and understand its architecture
- [ ] Tokenize protein sequences for ESM
- [ ] Calculate log-likelihood ratios for mutations
- [ ] Interpret LLR values (negative = disfavored)
- [ ] Run ensemble predictions for robustness
- [ ] Visualize mutation effects

### Your lab notebook should include:

- [ ] LLR scores for all target mutations
- [ ] Ensemble predictions with uncertainty
- [ ] Visualization of mutation effects
- [ ] Comparison with known clinical data
- [ ] Notes on which mutations are predicted deleterious

---

## 🤔 Reflection Questions

Answer these in your lab notebook:

1. **Model Limitations**: ESM-1v predicts evolutionary constraint, not drug resistance directly. Why might a mutation with moderately negative LLR still cause resistance?

2. **Ensemble Uncertainty**: Which mutations showed highest uncertainty (std) across models? What might cause model disagreement?

3. **Clinical Application**: How would you use ESM-1v predictions in a drug resistance screening pipeline? What other data would you combine it with?

---

## 📖 Further Reading

- [ESM-1v Paper (Meier et al., 2021)](https://www.biorxiv.org/content/10.1101/2021.07.09.450648v2) - Original bioRxiv preprint
- [fair-esm GitHub](https://github.com/facebookresearch/esm) - Official ESM repository
- [ESM Atlas](https://esmatlas.com/) - Large-scale structure predictions using ESM
- [Protein Language Models Review](https://www.nature.com/articles/s41587-022-01432-w) - Comprehensive overview

---

## 🔗 Connection to Research

ESM-1v enables zero-shot mutation effect prediction that complements structure-based methods:

- **Drug Resistance Surveillance**: Screen patient sequences for known and novel resistance mutations
- **Protein Engineering**: Guide directed evolution by predicting mutation tolerability
- **Clinical Variant Interpretation**: Assess pathogenicity of patient-specific variants
- **Evolutionary Analysis**: Understand constraints on drug targets

In the next talktorial (AI-PSCI-014), you'll learn to **integrate all the AI tools** (ESMFold, AlphaFold2, ESM-1v) into a unified pipeline for comprehensive mutation analysis.

---

*AI-PSCI-013 Complete. Proceed to AI-PSCI-014: Building an End-to-End Drug Discovery Pipeline.*